In [10]:
import pandas as pd
from pandas.io import gbq

RAW = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRhcAWkWXIjp2XVAsnTLw13QGg6Ot9D_HBf_FMCA42qIWf034T8oKOgV6cTBJS29tfJRPHPyQ4DQJ6s/pub?gid=1990926458&single=true&output=csv"

SCHEMA = {
    "project" : "object",
    "start" : "datetime64[ns]",
    "finish" : "datetime64[ns]",
    "project_id" : "object",
    "preparation_start_date" : "datetime64[ns]",
    "ready_to_deploy_date" : "datetime64[ns]",
    
}

df = pd.read_csv(RAW)

df = df.rename({"Timestamp":"last_record"}, axis=1)
df["Date Leads Acquired"] = df["Date Leads Acquired"].astype('datetime64[ns]')
df["Delegation Date"] = df["Delegation Date"].astype('datetime64[ns]')
df["Main Activity Start Date"] = df["Main Activity Start Date"].astype('datetime64[ns]')
df["Main Activity End Date"] = df["Main Activity End Date"].astype('datetime64[ns]')
df["Final Report Submitted Date"] = df["Final Report Submitted Date"].astype('datetime64[ns]')
df["Project Archived Date"] = df["Project Archived Date"].astype('datetime64[ns]')

df_drop = df.drop(["Project Name", "Date Leads Acquired", 
                    "Delegation Date", "Main Activity Start Date",
                    "Main Activity End Date", "Final Report Submitted Date", 
                    "Project Archived Date"], axis=1)

def phase(start, finish, phase):
    df_phase = df[["Project Name", start, finish]]
    df_phase = pd.concat([df_phase, df_drop], axis=1)
    df_phase = df_phase.dropna(subset=["Project Name"])
    df_phase["Phase"] = phase
    df_phase.rename(columns = {"Project Name":"Project", start:"Start", finish:"Finish"}, inplace=True)
    df_phase["Start"] = pd.to_datetime(df_phase["Start"])
    df_phase["Finish"] = pd.to_datetime(df_phase["Finish"])
    return df_phase

#Create Phase Columns
df_development = phase(start="Date Leads Acquired", finish="Delegation Date", phase="Development")
df_preparation = phase(start="Delegation Date", finish="Main Activity Start Date", phase="Preparation")
df_active = phase(start="Main Activity Start Date", finish="Main Activity End Date", phase="Active")
df_reporting = phase(start="Main Activity End Date", finish="Final Report Submitted Date", phase="Reporting")
df_closing = phase(start="Final Report Submitted Date", finish="Project Archived Date", phase="Closing")
df_gantt = pd.concat(
                [df_development, df_preparation, df_active, df_reporting, df_closing], 
                ignore_index=True)

df_gantt["timestamp"] = pd.Timestamp('now')

#cleaning columns
replace = {
    ' ' : '_',
    '-' : '_',
    '.' : '_',
    '?' : '',
    '[' : '',
    ']' : '',
    '(' : '',
    ')' : ''
}

def clean_col (col, clean_col):
  col.columns = col.columns.str.lower()
  for i, j in clean_col.items():
    col.columns = col.columns.str.replace(i, j)
  return col

df_2 = clean_col(df_gantt, replace)

df_2 = df_2.astype(SCHEMA)
pd.set_option('display.max_columns', None)
df_2.dtypes

/tmp/ipykernel_16938/3066250415.py:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  col.columns = col.columns.str.replace(i, j)
/tmp/ipykernel_16938/3066250415.py:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  col.columns = col.columns.str.replace(i, j)
/tmp/ipykernel_16938/3066250415.py:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  col.columns = col.columns.str.replace(i, j)
/tmp/ipykernel_16938/3066250415.py:66: FutureWarning: The default value of regex will change from True to False in a future version. In add

project                           object
start                     datetime64[ns]
finish                    datetime64[ns]
project_id                        object
preparation_start_date    datetime64[ns]
                               ...      
cogs                             float64
cash_in                          float64
nsm                              float64
phase                             object
timestamp                 datetime64[ns]
Length: 160, dtype: object